In [25]:
#First 6 cells of code is copied from Q3_Assignment3 submission by Dhvani Shah
#Only extra part added is input of end effector velocity
import numpy as np
import math
import matplotlib.pyplot as plt

In [18]:
#degrees of freedom
DOFs = int(input("Enter degrees of freedom"))

theta = np.zeros((DOFs))
alpha = np.zeros((DOFs))
a = np.zeros((DOFs))
d = np.zeros((DOFs))

for i in range(0, DOFs):
  print("data for link", i)
  theta[i] = math.pi*float(input("Theta"))/180
  alpha[i] = math.pi*float(input("Alpha"))/180
  a[i] = float(input("a"))
  d[i] = float(input("d"))

velo = np.array(list(map(float, input("Enter end effector velocity in the form (vx vy vz wx wy wz)").split()))).reshape((6,1))

Enter degrees of freedom3
data for link 0
Theta0
Alpha-90
a0
d5
data for link 1
Theta-90
Alpha-90
a0
d5
data for link 2
Theta90
Alpha-90
a0
d5
Enter end effector velocity in the form (vx vy vz wx wy wz)1 1 1 0 0 0


In [19]:
joint = [0]*DOFs
for i in range(0, DOFs):
  print("Type of joint", i+1)
  val = input()
  if val == "p":
    joint[i] = "p"
  else:
    joint[i] = "r"

Type of joint 1
p
Type of joint 2
p
Type of joint 3
p


In [20]:
#Transformation Matrix
temp = np.identity((4))
T = np.zeros((DOFs, 4, 4))

for i in range(0, DOFs):
  costheta = math.cos(theta[i])
  sintheta = math.sin(theta[i])
  cosalpha = math.cos(alpha[i])
  sinalpha = math.sin(alpha[i])

  T[i, :, :] = [[costheta, -sintheta*cosalpha, sintheta*sinalpha, a[i]*costheta], 
                [sintheta, costheta*cosalpha, -costheta*sinalpha, a[i]*sintheta], 
                [0, sinalpha, cosalpha, d[i]], 
                [0, 0, 0, 1]]

print(T)

[[[ 1.00000000e+00 -0.00000000e+00 -0.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00  6.12323400e-17  1.00000000e+00  0.00000000e+00]
  [ 0.00000000e+00 -1.00000000e+00  6.12323400e-17  5.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 [[ 6.12323400e-17  6.12323400e-17  1.00000000e+00  0.00000000e+00]
  [-1.00000000e+00  3.74939946e-33  6.12323400e-17 -0.00000000e+00]
  [ 0.00000000e+00 -1.00000000e+00  6.12323400e-17  5.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]

 [[ 6.12323400e-17 -6.12323400e-17 -1.00000000e+00  0.00000000e+00]
  [ 1.00000000e+00  3.74939946e-33  6.12323400e-17  0.00000000e+00]
  [ 0.00000000e+00 -1.00000000e+00  6.12323400e-17  5.00000000e+00]
  [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]]


In [21]:
temp = np.identity((4))
Z = [0]*(DOFs + 1)
Z[0] = np.array([[0], [0], [1]])
O = [0]*(DOFs + 1)
O[0] = np.array([0, 0, 0])

for i in range(DOFs):
  temp = np.matmul(temp,T[i])
  Z[i+1] = np.matmul(temp[0:3, 0:3],Z[0])
  O[i+1] = temp[0:3, 3].T

print(temp)

[[ 6.12323400e-17 -1.00000000e+00  0.00000000e+00  5.00000000e+00]
 [-1.00000000e+00 -6.12323400e-17  3.74939946e-33  5.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00 -1.00000000e+00  5.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [22]:
J = [0]*DOFs
for i in range(DOFs):
  if joint[i] == 'p':
    J[i] = np.concatenate((Z[i], np.array([[0,0,0]]).T), axis=0)

  else:
    mat = np.cross(Z[i].T, (O[DOFs]-O[i]).T)
    J[i]=np.concatenate((mat.T,Z[i]),axis=0)

jacobian = J[0]
for i in range(1, DOFs):
  jacobian = np.concatenate((jacobian,J[i]),axis=1)

print("Jacobian matrix=")
print(jacobian)

Jacobian matrix=
[[ 0.000000e+00  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  6.123234e-17]
 [ 1.000000e+00  6.123234e-17 -6.123234e-17]
 [ 0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00]]


In [23]:
#Jacobian Inverse
jacobian_inv = np.linalg.pinv(jacobian)

print(jacobian_inv)
#Joint velocities
joint_velo = np.matmul(jacobian_inv, velo)
print(joint_velo)

[[0.00000000e+00 0.00000000e+00 1.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [2.33240805e-17 1.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [1.00000000e+00 3.55080835e-18 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]
[[1.]
 [1.]
 [1.]]
